In [ ]:
import os
from os import path as op

import matplotlib.pyplot as plt
from mne import read_evokeds, combine_evoked
from mne.epochs import combine_event_ids
from mne.utils.numerics import grand_average
from mnefun import read_params as rp
from pathlib import Path
import numpy as np
import seaborn as sns
from os import path as op
import pandas as pd
import janitor
import pandas_flavor as pf
from pandas_profiling import ProfileReport

In [ ]:
### Sample charactersitics & findings 

- Strong correlations betweeen SES and parental education levels.
#todo
    [] Get REFs for impact of parental education on SES. 

In [ ]:
static = '/home/ktavabi/Github/badbaby/badbaby/static'
df = pd.read_excel(op.join(static, 'meg_covariates.xlsx'), sheet_name='mmn')
df.info()
covars = (
    pd.read_excel(op.join(static, 'meg_covariates.xlsx'), sheet_name='mmn')
    .clean_names()
    .select_columns(['subjid', 'ses', 'age', 'gender', 'headsize',
                     'maternaledu', 'paternaledu', 'maternalethno', 'paternalethno'])
    .rename_columns({'subjid': 'id'})
    .encode_categorical(columns=['gender', 'maternalethno', 'paternalethno'])
    .bin_numeric(
        from_column_name='maternaledu',
        to_column_name='Education (yrs)',
        num_bins=3,
        labels=['9-12', '13-17', '18-21'])
)
covars.describe()
profile = ProfileReport(covars, title="MEG covariates profile").to_widgets()
sns.set_theme(style="ticks", color_codes=True)
# plot distributions of SES amongst genders as a function of mother's years-edu
sns.catplot(x="Education (yrs)", y="ses",
            hue="gender", kind="bar", data=covars)
sns.catplot(y="maternalethno", hue="gender",
            kind="count", data=covars)  # so much for equity & diversity
sns.lmplot(y="ses", x="maternaledu", data=covars, x_estimator=np.mean)
sns.lmplot(y="age", x="headsize", data=covars, x_estimator=np.mean)

In [ ]:
### Grand averaging

In [ ]:
prms = rp(op.join("/home/ktavabi/Github/badbaby/badbaby", "processing", "badbaby.yml"))
wrkDir = '/media/ktavabi/ALAYA/data/ilabs/badbaby'
lp = prms.lp_cut

In [ ]:
# averaging gist c/o Larson
conditions = ['standard', 'deviant']
evoked_dict = dict((key, list()) for key in conditions)
for condition in conditions:
    for subject in prms.subjects:
        fname = op.join(wrkDir, subject, 'inverse',
                        f'Oddball_{lp}-sss_eq_{subject}-ave.fif')
        evoked_dict[condition].append(read_evokeds(fname, condition))
evoked = combine_evoked(
    [grand_average(evoked_dict[condition])
     for condition in conditions], weights=[-1, 1])
evoked.pick_types(meg=True)
evoked.plot_joint()

In [ ]:
ERFS = dict()
for kk in evoked_dict.keys():
    ERFS[kk] = mne.grand_average(evoked_dict[kk])
mne.viz.plot_compare_evokeds(ERFS, combine='gfp', ci=0.9)
ERF = mne.combine_evoked([ERFS[cc] for cc in conditions], weights=[-1, 1])
ERF.pick_types(meg=True)
ERF.plot_joint()

In [ ]:
events = {'standard': [], 'deviant': []}
for condition in events.keys():
    evs = list()
    for si, subj in enumerate(prms.subjects):
        print(' %s' % subj)
        filename = op.join(wrkDir, subj, 'inverse',
                                'Oddball_%d-sss_eq_%s-ave.fif' % (lp, subj))
        evs.append(read_evokeds(filename, condition=condition,
                       baseline=(None, 0)))
    # do grand averaging
    print('  Doing %s averaging.' % condition)
    events[condition].append(grand_average(evs))